In [1]:
cd /content/drive/My Drive/Colab Notebooks/esrgan

/content/drive/My Drive/Colab Notebooks/esrgan


In [2]:
import os
import numpy as np
import math
import itertools
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR

from models import *
from datasets import *

import torch.nn as nn
import torch.nn.functional as F
import torch

In [3]:
os.makedirs("results", exist_ok=True)

In [5]:
dataset_name = "/content/drive/My Drive/Colab Notebooks/esrgan/archive/test_data_images/test_data_images/images"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cuda = torch.cuda.is_available()
hr_shape = (224, 224)
channels = 3
num_res_blocks = 23
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

In [6]:
generator = generator = GeneratorRRDB(channels, filters=64, num_res_blocks=num_res_blocks).to(device)
generator = generator.cuda()
generator.load_state_dict(torch.load("saved_models/generator_200.pth"))

<All keys matched successfully>

In [7]:
test_dataloader = DataLoader(
    ImageDataset(dataset_name, hr_shape=hr_shape),
    batch_size=3,
    shuffle=False,
    num_workers=8,
)

In [8]:
batch = iter(test_dataloader)
imgs = next(batch)
imgs_lr = Variable(imgs["lr"].type(Tensor))
imgs_hr = Variable(imgs["hr"].type(Tensor))

generator.eval()
gen_hr = generator(imgs_lr)

imgs_lr = nn.functional.interpolate(imgs_lr, scale_factor=4)
imgs_hr = nn.functional.interpolate(imgs_hr, scale_factor=1)

img_grid = denormalize(torch.cat((imgs_lr, gen_hr, imgs_hr), -1))


save_image(img_grid, "results/sample.png",  normalize=False)

In [9]:
for i, imgs in enumerate(test_dataloader, 1):
  imgs_lr = Variable(imgs["lr"].type(Tensor))
  imgs_hr = Variable(imgs["hr"].type(Tensor))

  generator.eval()
  gen_hr = generator(imgs_lr)

  imgs_lr = nn.functional.interpolate(imgs_lr, scale_factor=4)
  imgs_hr = nn.functional.interpolate(imgs_hr, scale_factor=1)

  img_grid = denormalize(torch.cat((imgs_lr, gen_hr, imgs_hr), -1))

  save_image(img_grid, "results/%d.png" % i, normalize=False)

  if i == 10:
    break